In [21]:
import pandas as pd
import glob

# Research Question

How does combine performance affect the draft stock of an NFL athlete?

# Hypothesis

The better that NFL draftee's combine performance will cause them to be drafted higher in relation to other players of the same position. 

In [15]:
csv_files = glob.glob('./combine-data/*.csv')
draft_df = pd.read_csv('./draft-data/nfl_draft_prospects.csv')
comb_df = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(file)
    comb_df = pd.concat([comb_df, df])

select_df = draft_df[['draft_year', 'player_name', 'school', 'pick', 'overall', 'round']]

In [19]:
merged_df = pd.merge(comb_df, select_df, left_on='Player', right_on='player_name', how='inner')

In [24]:
comb_df.head(10)

,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,John Abraham,OLB,South Carolina,6-4,252.0,4.55,NaN,NaN,NaN,NaN,NaN
1,Shaun Alexander,RB,Alabama,6-0,218.0,4.58,NaN,NaN,NaN,NaN,NaN
2,Darnell Alford,OT,Boston Col.,6-4,334.0,5.56,25.0,23.0,94.0,8.48,4.98
3,Kyle Allamon,TE,Texas Tech,6-2,253.0,4.97,29.0,NaN,104.0,7.29,4.49
4,Rashard Anderson,CB,Jackson State,6-2,206.0,4.55,34.0,NaN,123.0,7.18,4.15
5,Jake Arians,K,Ala-Birmingham,5-10,202.0,NaN,NaN,NaN,NaN,NaN,NaN
6,LaVar Arrington,OLB,Penn State,6-3,250.0,4.53,NaN,NaN,NaN,NaN,NaN
7,Corey Atkins,OLB,South Carolina,6-0,237.0,4.72,31.0,21.0,112.0,7.96,4.39
8,Kyle Atteberry,K,Baylor,6-0,167.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Reggie Austin,CB,Wake Forest,5-9,175.0,4.44,35.0,17.0,119.0,7.03,4.14


In [22]:
select_df.head(10)

,draft_year,player_name,school,pick,overall,round
0,1967,Bubba Smith,Michigan State,1.0,1.0,1.0
1,1967,Clinton Jones,Michigan State,2.0,2.0,1.0
2,1967,Steve Spurrier,Florida,3.0,3.0,1.0
3,1967,Bob Griese,Purdue,4.0,4.0,1.0
4,1967,George Webster,Michigan State,5.0,5.0,1.0
5,1967,Floyd Little,Syracuse,6.0,6.0,1.0
6,1967,Mel Farr,UCLA,7.0,7.0,1.0
7,1967,Gene Washington,Michigan State,8.0,8.0,1.0
8,1967,Bob Hyland,Boston College,9.0,9.0,1.0
9,1967,Loyd Phillips,Arkansas,10.0,10.0,1.0


In [23]:
merged_df.head(10)

,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,draft_year,player_name,school,pick,overall,round
0,John Abraham,OLB,South Carolina,6-4,252.0,4.55,NaN,NaN,NaN,NaN,NaN,2000,John Abraham,South Carolina,13.0,13.0,1.0
1,Shaun Alexander,RB,Alabama,6-0,218.0,4.58,NaN,NaN,NaN,NaN,NaN,2000,Shaun Alexander,Alabama,19.0,19.0,1.0
2,Darnell Alford,OT,Boston Col.,6-4,334.0,5.56,25.0,23.0,94.0,8.48,4.98,2000,Darnell Alford,Boston College,22.0,188.0,6.0
3,Rashard Anderson,CB,Jackson State,6-2,206.0,4.55,34.0,NaN,123.0,7.18,4.15,2000,Rashard Anderson,Jackson State,23.0,23.0,1.0
4,LaVar Arrington,OLB,Penn State,6-3,250.0,4.53,NaN,NaN,NaN,NaN,NaN,2000,LaVar Arrington,Penn State,2.0,2.0,1.0
5,Reggie Austin,CB,Wake Forest,5-9,175.0,4.44,35.0,17.0,119.0,7.03,4.14,2000,Reggie Austin,Wake Forest,31.0,125.0,4.0
6,Mark Baniewicz,OT,Syracuse,6-6,312.0,5.34,28.0,20.0,96.0,7.72,4.73,2000,Mark Baniewicz,Syracuse,41.0,247.0,7.0
7,Rashidi Barnes,S,Colorado,6-0,208.0,4.62,35.0,10.0,114.0,6.92,4.32,2000,Rashidi Barnes,Colorado,19.0,225.0,7.0
8,David Barrett,CB,Arkansas,5-10,199.0,4.44,37.5,16.0,116.0,6.81,4.04,2000,David Barrett,Arkansas,8.0,102.0,4.0
9,William Bartee,CB,Oklahoma,6-1,192.0,4.43,38.5,NaN,124.0,6.70,3.99,2000,William Bartee,Oklahoma,23.0,54.0,2.0
